#data from VGchartz to get sell data

https://www.vgchartz.com/gamedb/



In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import os

In [2]:
# scrape the list of genre

# URL to be scraped
url = 'https://www.vgchartz.com/gamedb/?page='

# 發送 HTTP 請求
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response = requests.get(url, headers=headers)


# Parse HTML webpage source using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Scrape the list of genre
genre_list = []
result_select = soup.find('select', {'name':'genre'})
result_options = result_select.find_all('option')
for result in result_options:
    if result['value'] != '':
        genre_list.append(result['value'])
genre_list

['Action',
 'Action-Adventure',
 'Adventure',
 'Board Game',
 'Education',
 'Fighting',
 'Misc',
 'MMO',
 'Music',
 'Party',
 'Platform',
 'Puzzle',
 'Racing',
 'Role-Playing',
 'Sandbox',
 'Shooter',
 'Simulation',
 'Sports',
 'Strategy',
 'Visual Novel']

In [3]:
#!pip install html5lib

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
import os
import random
import re
import math
from datetime import date, datetime
from lxml import html

# 创建session来保持cookies
session = requests.Session()

# 更新headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Cache-Control': 'max-age=0',
    'Referer': 'https://www.vgchartz.com/gamedb/'
}

# 游戏类型列表
genre_list = [
    "Action",
    "Adventure",
    "Action-Adventure",
    "Board+Game",
    "Education",
    "Fighting",
    "Misc",
    "MMO",
    "Music",
    "Party",
    "Platform",
    "Puzzle",
    "Racing",
    "Role-Playing",
    "Sandbox",
    "Shooter",
    "Simulation",
    "Sports",
    "Strategy",
    "Visual+Novel"
]

def get_page_with_retry(url, max_retries=3):
    """使用重试机制获取页面"""
    for i in range(max_retries):
        try:
            response = session.get(url, headers=headers, timeout=15)
            if response.status_code == 200:
                return response
            print(f"请求状态码: {response.status_code}")
            time.sleep(random.uniform(3, 6))
        except Exception as e:
            print(f"尝试 {i+1}/{max_retries} 失败: {str(e)}")
            if i < max_retries - 1:
                time.sleep(random.uniform(3, 6))
    return None

def scrape_vgchartz():
    """主要的爬取函数"""
    # 创建空的DataFrame
    all_games_data = []
    
    # 遍历每个游戏类型
    for genre in genre_list:
        print(f"\n开始处理类型: {genre}")
        page_num = 1
        
        # 获取该类型的第一页，用于确定总页数
        base_url = f"https://www.vgchartz.com/games/games.php?page=1&results=200&name=&console=&keyword=&publisher=&genre={genre}&order=Sales&ownership=Both&boxart=Both&banner=Both&showdeleted=&region=All&goty_year=&developer=&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&showcriticscore=1&showuserscore=1&showshipped=1"
        
        response = get_page_with_retry(base_url)
        if not response:
            print(f"无法获取 {genre} 的数据，跳过")
            continue
        
        # 使用lxml解析HTML
        tree = html.fromstring(response.content)
        
        # 获取该类型的总结果数
        results_selector = '//*[@id="generalBody"]/table[1]/tr[1]/th[1]/text()'
        results = tree.xpath(results_selector)[0]
        results = results.replace(",", "")
        results_pat = r'([0-9]{1,9})'
        total_results = int(re.search(results_pat, results).group(1))
        
        # 计算总页数
        last_page = math.ceil(total_results / 200)
        print(f"共找到 {total_results} 条记录，{last_page} 页")
        
        # 定义XPath选择器，与test.py保持一致
        IMAGE_SELECTOR = './/td[2]/div/a/div/img/@src'
        TITLE_SELECTOR = './/td[3]/a/text()'
        CONSOLE_SELECTOR = './/td[4]/img/@alt'
        PUBLISHER_SELECTOR = './/td[5]/text()'
        DEVELOPER_SELECTOR = './/td[6]/text()'
        VGSCORE_SELECTOR = './/td[7]/text()'
        CRITIC_SELECTOR = './/td[8]/text()'
        USER_SELECTOR = './/td[9]/text()'
        TOTALSHIPPED_SELECTOR = './/td[10]/text()'
        TOTALSALES_SELECTOR = './/td[11]/text()'
        NASALES_SELECTOR = './/td[12]/text()'
        PALSALES_SELECTOR = './/td[13]/text()'
        JPSALES_SELECTOR = './/td[14]/text()'
        OTHER_SELECTOR = './/td[15]/text()'
        RELEASEDATE_SELECTOR = './/td[16]/text()'
        UPDATE_SELECTOR = './/td[17]/text()'
        
        # 循环处理每一页
        while page_num <= last_page:
            print(f"  正在处理第 {page_num}/{last_page} 页...")
            
            # 如果不是第一页，重新获取页面
            if page_num > 1:
                url = f"https://www.vgchartz.com/games/games.php?page={page_num}&results=200&name=&console=&keyword=&publisher=&genre={genre}&order=Sales&ownership=Both&boxart=Both&banner=Both&showdeleted=&region=All&goty_year=&developer=&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&showcriticscore=1&showuserscore=1&showshipped=1"
                response = get_page_with_retry(url)
                if not response:
                    print(f"获取第 {page_num} 页失败，跳过")
                    page_num += 1
                    continue
                tree = html.fromstring(response.content)
            
            # 获取表格中的所有行
            rows = tree.xpath('//*[@id="generalBody"]/table[1]/tr')
            
            # 从第4行开始处理数据行（跳过表头和导航栏）
            for row in rows[4:]:
                try:
                    # 使用XPath提取每一列的数据
                    img = row.xpath(IMAGE_SELECTOR)
                    title = row.xpath(TITLE_SELECTOR)
                    console = row.xpath(CONSOLE_SELECTOR)
                    publisher = row.xpath(PUBLISHER_SELECTOR)
                    developer = row.xpath(DEVELOPER_SELECTOR)
                    vg_score = row.xpath(VGSCORE_SELECTOR)
                    critic_score = row.xpath(CRITIC_SELECTOR)
                    user_score = row.xpath(USER_SELECTOR)
                    total_shipped = row.xpath(TOTALSHIPPED_SELECTOR)
                    total_sales = row.xpath(TOTALSALES_SELECTOR)
                    na_sales = row.xpath(NASALES_SELECTOR)
                    pal_sales = row.xpath(PALSALES_SELECTOR)
                    jp_sales = row.xpath(JPSALES_SELECTOR)
                    other_sales = row.xpath(OTHER_SELECTOR)
                    release_date = row.xpath(RELEASEDATE_SELECTOR)
                    last_update = row.xpath(UPDATE_SELECTOR)
                    
                    # 只有当标题不为空时才添加记录
                    if title and len(title) > 0:
                        game_data = {
                            'img': img[0] if img else '',
                            'title': title[0] if title else '',
                            'console': console[0] if console else '',
                            'publisher': publisher[0].strip() if publisher else '',
                            'developer': developer[0].strip() if developer else '',
                            'vg_score': vg_score[0].strip() if vg_score else '',
                            'critic_score': critic_score[0].strip() if critic_score else '',
                            'user_score': user_score[0].strip() if user_score else '',
                            'total_shipped': total_shipped[0].strip() if total_shipped else '',
                            'total_sales': total_sales[0].strip() if total_sales else '',
                            'na_sales': na_sales[0].strip() if na_sales else '',
                            'pal_sales': pal_sales[0].strip() if pal_sales else '',
                            'jp_sales': jp_sales[0].strip() if jp_sales else '',
                            'other_sales': other_sales[0].strip() if other_sales else '',
                            'release_date': release_date[0].strip() if release_date else '',
                            'last_update': last_update[0].strip() if last_update else '',
                            'genre': genre.replace('+', ' ')
                        }
                        all_games_data.append(game_data)
                except Exception as e:
                    print(f"处理行数据时出错: {str(e)}")
                    continue
            
            page_num += 1
            # 避免请求过快
            time.sleep(random.uniform(3, 6))
    
    # 转换为DataFrame
    games_df = pd.DataFrame(all_games_data)
    
    # 数据清洗
    # 替换所有"N/A"为np.nan
    games_df = games_df.replace("N/A", np.nan)
    
    # 清洗数值型字段
    sales_columns = ["na_sales", "jp_sales", "pal_sales", "other_sales", 
                      "total_sales", "total_shipped", "vg_score", "user_score", "critic_score"]
    
    # 定义清洗数值的函数
    def clean_nums(column):
        # 去除'm'后缀并转换为浮点数
        if games_df[column].dtype == 'object':
            games_df[column] = games_df[column].str.replace('m', '')
            games_df[column] = pd.to_numeric(games_df[column], errors='coerce')
    
    # 应用到所有销量列
    for column in sales_columns:
        if column in games_df.columns:
            clean_nums(column)
    
    # 清洗日期字段
    # 月份数字映射
    month_to_num = {
        'Sep': 9, 'Jul': 7, 'Oct': 10, 'Mar': 3, 'Dec': 12, 'Feb': 2,
        'Nov': 11, 'Jun': 6, 'Aug': 8, 'May': 5, 'Apr': 4, 'Jan': 1
    }
    
    # 定义清洗日期的函数
    def clean_dates(text):
        if pd.isna(text):
            return np.nan
        
        try:
            day_pat = r"([0-9]{2})(?=[a-z]{2})"
            month_pat = r"([A-Z][a-z]{2})"
            year_pat = r"([0-9]{2}(?![a-z]{2}))"
            
            day = int(re.search(day_pat, text).group(1))
            month = month_to_num[re.search(month_pat, text).group(1)]
            year = re.search(year_pat, text).group(1)
            
            if int(year[0]) >= 7:
                year = int("19" + year)
            else:
                year = int("20" + year)
                
            return datetime(year, month, day)
        except:
            return np.nan
    
    # 应用到日期列
    for column in ["last_update", "release_date"]:
        if column in games_df.columns:
            games_df[column] = games_df[column].apply(clean_dates)
    
    # 替换1970年的日期为np.nan
    if 'release_date' in games_df.columns:
        mask = games_df["release_date"].dt.year == 1970
        games_df.loc[mask, "release_date"] = np.nan
    
    # 重新排序列
    desired_order = ["img", "title", "console", "genre", "publisher", "developer", 
                     "vg_score", "critic_score", "user_score", "total_shipped", 
                     "total_sales", "na_sales", "jp_sales", "pal_sales", "other_sales", 
                     "release_date", "last_update"]
    
    # 确保只包括存在的列
    columns_to_use = [col for col in desired_order if col in games_df.columns]
    games_df = games_df[columns_to_use]
    
    return games_df

def main():
    print("开始爬取VGChartz游戏数据...")
    games_df = scrape_vgchartz()
    
    if games_df is not None and not games_df.empty:
        print("\n数据收集完成！")
        print(f"总共收集了 {len(games_df)} 条游戏数据")
        print("\n数据框信息:")
        games_df.info()
        print("\n前5行数据:")
        print(games_df.head())
        
        # 保存数据
        try:
            today_date_string = f"{date.today().month}_{date.today().day}_{date.today().year}"
            filename = f'vgchartz-{today_date_string}.csv'
            games_df.to_csv(filename, index=False, encoding='utf-8')
            print(f"File saved as vgchartz-{today_date_string}.csv")
            print(f"{games_df['title'].count()} game records retrieved.")
        except Exception as e:
            print(f"\n保存数据时出错: {str(e)}")
    else:
        print("\n未收集到任何数据")

if __name__ == "__main__":
    main()

开始爬取VGChartz游戏数据...

开始处理类型: Action
共找到 8897 条记录，45 页
  正在处理第 1/45 页...
  正在处理第 2/45 页...
  正在处理第 3/45 页...
  正在处理第 4/45 页...
  正在处理第 5/45 页...
  正在处理第 6/45 页...
  正在处理第 7/45 页...
  正在处理第 8/45 页...
  正在处理第 9/45 页...
  正在处理第 10/45 页...
  正在处理第 11/45 页...
  正在处理第 12/45 页...
  正在处理第 13/45 页...
  正在处理第 14/45 页...
  正在处理第 15/45 页...
  正在处理第 16/45 页...
  正在处理第 17/45 页...
  正在处理第 18/45 页...
  正在处理第 19/45 页...
  正在处理第 20/45 页...
  正在处理第 21/45 页...
  正在处理第 22/45 页...
  正在处理第 23/45 页...
  正在处理第 24/45 页...
  正在处理第 25/45 页...
  正在处理第 26/45 页...
  正在处理第 27/45 页...
  正在处理第 28/45 页...
  正在处理第 29/45 页...
  正在处理第 30/45 页...
  正在处理第 31/45 页...
  正在处理第 32/45 页...
  正在处理第 33/45 页...
  正在处理第 34/45 页...
  正在处理第 35/45 页...
  正在处理第 36/45 页...
  正在处理第 37/45 页...
  正在处理第 38/45 页...
  正在处理第 39/45 页...
  正在处理第 40/45 页...
  正在处理第 41/45 页...
  正在处理第 42/45 页...
  正在处理第 43/45 页...
  正在处理第 44/45 页...
  正在处理第 45/45 页...

开始处理类型: Adventure
共找到 6483 条记录，33 页
  正在处理第 1/33 页...
  正在处理第 2/33 页...
  正在处理第 3/33 页...
  正在处理第 4

In [5]:
#!pip install fake-useragent html5lib